# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import joblib, pickle

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.datapath import DataPath
from azureml.data.dataset_factory import TabularDatasetFactory

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.26.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [16]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'mobile_price_prediction'

experiment=Experiment(ws, experiment_name)

In [17]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster = 'automl-cluster'

try:
    comp_trget = ComputeTarget(workspace=ws, name=compute_cluster)
    print('Cluster already exists')
except ComputeTargetException:
    compute_confg = AmlCompute.provisioning_configuration(vm_size="STANDARD_D3_V2",max_nodes=4)
    comp_trget = ComputeTarget.create(ws,compute_cluster,compute_confg)

Cluster already exists


In [18]:
input_data = pd.read_csv("https://raw.githubusercontent.com/valeriapolozun/machine_learning_azure_capstone/803e04124532b8921a0b19d70aa71abaccf74a1f/Mobile_Price_Classification.csv")

In [19]:
#input_data=pd.read_csv('Mobile_Price_Classification.csv')

In [20]:
input_data

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0.0,1,0,7,0.6,188.0,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1.0,0,1,53,0.7,136.0,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1.0,2,1,41,0.9,145.0,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0.0,0,0,10,0.8,131.0,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0.0,13,1,44,0.6,141.0,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,996,1700,1.0,1.9,0,0,1,54.0,0.5,170,...,17,644,913,2121,14,8,15,1,1,0
2996,997,609,0.0,1.8,1,0,0,13.0,0.9,186,...,2,1152,1632,1933,8,1,19,0,1,1
2997,998,1185,0.0,1.4,0,1,1,8.0,0.5,80,...,12,477,825,1223,5,0,14,1,0,0
2998,999,1533,1.0,0.5,1,0,0,50.0,0.4,171,...,12,38,832,2509,15,11,6,0,1,0


In [21]:
# split data into training and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(input_data,test_size=0.2)
train_data.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
2971,972,1207,0.0,1.1,1,9,1,56.0,0.1,141,...,11,93,560,2293,12,4,19,1,1,0
1056,812,0,0.5,1.0,1,0,64,1.0,106.0,4,...,35,1309,3102,6,0,2,0,0,0,2
2848,849,1170,1.0,2.2,0,4,1,63.0,0.7,83,...,5,442,1905,2658,11,10,11,1,0,1
1217,1451,1,0.5,0.0,5,1,57,0.6,181.0,8,...,812,1456,3038,15,7,18,1,0,1,3
2505,506,532,0.0,0.5,0,7,0,51.0,0.6,180,...,14,406,1249,1525,7,6,2,0,1,1


In [22]:
if "data" not in os.listdir():
    #os.mkdir("./training")if "training" not in os.listdir():
    os.mkdir("./data")

In [23]:
# Save training data to a csv file 
train_data.to_csv("data/mobile_price_traindata.csv", index=False)

In [24]:
data_store = ws.get_default_datastore()
data_store

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-0b49a19a-37f6-4737-ab05-a7102ad81896",
  "account_name": "mlstrg143627",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [25]:
data_store.upload(src_dir='data/',target_path='pricing', overwrite=True, show_progress=True)


Uploading an estimated of 1 files
Uploading data/mobile_price_traindata.csv
Uploaded data/mobile_price_traindata.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_c2a5613704234991afd9fc3db2de458e

In [26]:
train_data = Dataset.Tabular.from_delimited_files(path = [(data_store, 'pricing/mobile_price_traindata.csv')])

In [27]:
train_data

{
  "source": [
    "('workspaceblobstore', 'pricing/mobile_price_traindata.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [28]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             compute_target=comp_trget,
                             task = "classification",
                             training_data=train_data,
                             label_column_name="price_range",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             n_cross_validations=5,
                             **automl_settings
)

In [29]:
# TODO: Submit your experiment
#remote_run = experiment.submit(automl_config, show_output = True)
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
mobile_price_prediction,AutoML_4c763df7-6fab-4009-9e8c-7b581d7a4ce6,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?


TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [30]:
#remote_run = experiment.submit(automl_config, show_output = True)

from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
mobile_price_prediction,AutoML_4c763df7-6fab-4009-9e8c-7b581d7a4ce6,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

{'runId': 'AutoML_4c763df7-6fab-4009-9e8c-7b581d7a4ce6',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-04-27T13:41:20.350187Z',
 'endTimeUtc': '2021-04-27T14:05:44.912878Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"mobile_price_prediction","subscription_id":"6971f5ac-8af1-446e-8034-05acea24681f","resource_group":"aml-quickstarts-143627","workspace_name":"quick-starts-ws-143627","region":"southcentralus","compute_target":"automl-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":0.0,"v

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [31]:
best_automl, best_fit_model = remote_run.get_output()
print(best_fit_model)

Package:azureml-automl-runtime, training version:1.27.0.post1, current version:1.26.0
Package:azureml-core, training version:1.27.0, current version:1.26.0
Package:azureml-dataprep, training version:2.14.2, current version:2.13.2
Package:azureml-dataprep-native, training version:33.0.0, current version:32.0.0
Package:azureml-dataprep-rslex, training version:1.12.1, current version:1.11.2
Package:azureml-dataset-runtime, training version:1.27.0, current version:1.26.0
Package:azureml-defaults, training version:1.27.0, current version:1.26.0
Package:azureml-interpret, training version:1.27.0, current version:1.26.0
Package:azureml-mlflow, training version:1.27.0, current version:1.26.0
Package:azureml-pipeline-core, training version:1.27.0, current version:1.26.0
Package:azureml-telemetry, training version:1.27.0, current version:1.26.0
Package:azureml-train-automl-client, training version:1.27.0, current version:1.26.0
Package:azureml-train-automl-runtime, training version:1.27.0.post1,

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('stackensembleclassifier',
                 StackE...
                                         meta_learner=LogisticRegressionCV(Cs=10,
                                                                           class_weight=None,
                                                                           cv=None,
                                                                           dual=False,
                           

In [108]:
best_automl.get_properties()



{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Stack_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Stack_Ensemble__","objects":[{"module":"azureml.train.automl.stack_ensemble","class_name":"StackEnsemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'mobile_price_prediction\',\'compute_target\':\'automl-cluster\',\'subscription_id\':\'6971f5ac-8af1-446e-8034-05acea24681f\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_4c763df7-6fab-4009-9e8c-7b581d7a4ce6_37","experiment_name":"mobile_price_prediction","workspace_name":"quick-starts-ws-143627","subscription_id":"6971f5ac-8af1-446e-8034-05acea24681f","resource_group_name":"aml-quickstarts-143627"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '37',
 '_aml_system_scenario_identification'

In [32]:
#TODO: Save the best model
joblib.dump(best_fit_model, 'best_fit_automl_model.pkl')

['best_fit_automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [33]:
from azureml.core import Model

# Register model
best_automl.register_model(model_path='outputs/model.pkl', model_name='automl_model',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_automl.get_metrics()['accuracy']})



Model(workspace=Workspace.create(name='quick-starts-ws-143627', subscription_id='6971f5ac-8af1-446e-8034-05acea24681f', resource_group='aml-quickstarts-143627'), name=automl_model, id=automl_model:1, version=1, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.8004166666666667'})

In [39]:
# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

automl_model version: 1
	 Training context : Auto ML
	 Accuracy : 0.8004166666666667




In [40]:
from azureml.automl.core.shared import constants
best_automl.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

In [41]:
# create inference_config
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment


env=Environment.from_conda_specification(name='myenv',file_path='myenv.yml')
inference_config = InferenceConfig(entry_script="scoring.py",environment=env)

In [42]:

from azureml.core.webservice import Webservice, AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)


In [44]:
#deploy the model
service=Model.deploy(workspace=ws,
                    name="mobile-price-deploy-service",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)#deploy the model

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-27 14:25:51+00:00 Creating Container Registry if not exists.
2021-04-27 14:25:51+00:00 Registering the environment.
2021-04-27 14:25:52+00:00 Use the existing image.
2021-04-27 14:25:52+00:00 Generating deployment configuration.
2021-04-27 14:25:53+00:00 Submitting deployment to compute..
2021-04-27 14:25:58+00:00 Checking the status of deployment mobile-price-deploy-service..
2021-04-27 14:29:03+00:00 Checking the status of inference endpoint mobile-price-deploy-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [45]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')


service state: Healthy

scoring URI: 
http://f011a20f-089f-47a3-8454-b26bda1b946a.southcentralus.azurecontainer.io/score



In [58]:
print(input_data)
arr=json.loads(input_data)
print(type(arr))
print(arr)

{"data": [{"battery_power": 1043, "blue": 1, "clock_speed": 1.8, "dual_sim": 1, "fc": 14, "four_g": 0, "int_memory": 5, "m_dep": 0.1, "mobile_wt": 193, "n_cores": 3, "pc": 16, "px_height": 226, "px_width": 1412, "ram": 3476, "sc_h": 12, "sc_w": 7, "talk_time": 2, "three_g": 0, "touch_screen": 1, "wifi": 0}]}
<class 'dict'>
{'data': [{'battery_power': 1043, 'blue': 1, 'clock_speed': 1.8, 'dual_sim': 1, 'fc': 14, 'four_g': 0, 'int_memory': 5, 'm_dep': 0.1, 'mobile_wt': 193, 'n_cores': 3, 'pc': 16, 'px_height': 226, 'px_width': 1412, 'ram': 3476, 'sc_h': 12, 'sc_w': 7, 'talk_time': 2, 'three_g': 0, 'touch_screen': 1, 'wifi': 0}]}


In [59]:
data_json=json.loads(input_data)['data']
data=pd.DataFrame(data_json)
print(data)
print(data.shape)

   battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  m_dep  \
0           1043     1         1.80         1  14       0           5   0.10   

   mobile_wt  n_cores  pc  px_height  px_width   ram  sc_h  sc_w  talk_time  \
0        193        3  16        226      1412  3476    12     7          2   

   three_g  touch_screen  wifi  
0        0             1     0  
(1, 20)


In [54]:
data.shape

(1, 20)

In [109]:
import requests
import json

data={"data":
[{'battery_power':500,
  'blue': 1,
  'clock_speed':1.8,
  'dual_sim':1,
  'fc': 14,
  'four_g': 0,
  'int_memory': 5,
  'm_dep': 0.1,
  'mobile_wt': 193,
  'n_cores': 3,
  'pc': 16,
  'px_height':226,
   'px_width':1412,
   'ram':3476,
   'sc_h':12,
   'sc_w':7,
   'talk_time':2,
   'three_g':0,
   'touch_screen':1,
   'wifi':0
   }]
    }

input_data=json.dumps(data['data'])
#data_json=json.loads(input_data)['data']
#data=pd.DataFrame(data_json)


In [110]:


with open("data.json","w") as file:
    file.write(input_data)

headers={"Content-Type":"application/json"}

result=requests.post(scoring_uri, input_data, headers=headers)
print("The output is: ",result.json())


The output is:  [2]


TODO: In the cell below, print the logs of the web service and delete the service

In [99]:
print(service.get_logs())

2021-04-27T14:28:51,838191900+00:00 - rsyslog/run 
2021-04-27T14:28:51,846365600+00:00 - iot-server/run 
2021-04-27T14:28:51,865561700+00:00 - gunicorn/run 
rsyslogd: /azureml-envs/azureml_f8f5ff2f983718fa04a09abf22f98303/lib/libuuid.so.1: no version information available (required by rsyslogd)
2021-04-27T14:28:51,890584900+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_f8f5ff2f983718fa04a09abf22f98303/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_f8f5ff2f983718fa04a09abf22f98303/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_f8f5ff2f983718fa04a09abf22f98303/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_f8f5ff2f983718fa04a09abf22f98303/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml

In [ ]:

service.delete()
compute_target.delete()
model.delete()